In [1]:
import os
import sys
import random
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import traci
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
import itertools
import collections


## **SUMO Config**

In [2]:
def sumo_config(traffic_pattern="P1"):
    sumo_config = [
        "sumo",
        "-c", f"SUMO_networks/{traffic_pattern}/junction.sumocfg",
        "--step-length", "0.05",
        "--delay", "0",
        "--lateral-resolution", "0.1",
        "--start",
        "--no-warnings",
        "--no-step-log",
    ]
    return sumo_config

if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Declare SUMO_HOME env variable")

if not traci.isLoaded():
    traci.start(sumo_config())


def reset_env(pattern="P1"):
    if traci.isLoaded():
        traci.close()
    traci.start(sumo_config(pattern))


## **Simulation Setup**

In [3]:
# Global Variables for Simulation
action_space_size = 8
TRAFFIC_LIGHT_ID = "traffic_light"
DELTA_PHASE_DURATION = 6 
YELLOW_PHASE_DURATION = 4 
lane_detectors = [f'q{i+1}' for i in range(8)]
current_phase = 2


In [36]:
# Returns the queue length for each lane detector
def get_queue_length():
    return torch.tensor([
        traci.lanearea.getLastStepHaltingNumber(d) for d in lane_detectors
    ], dtype=torch.float)

# The current state is defined as queue length for each lane detector
def get_current_state():
    return get_queue_length()

# Simulate 20 seconds
def simulate_time(seconds=1):
    for _ in range(20 * seconds):
        traci.simulationStep()


In [37]:
def step(action):
    global current_phase
    if 2 * action == current_phase:
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, 2 * action)
        simulate_time(DELTA_PHASE_DURATION)
    else:
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, current_phase + 1)
        simulate_time(YELLOW_PHASE_DURATION)
        current_phase = 2 * action
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, 2 * action)
        simulate_time(DELTA_PHASE_DURATION)
        
    next_state = get_current_state()
    reward = -torch.sum(next_state)
    done = traci.simulation.getMinExpectedNumber() == 0
    return next_state, reward, done, next_state.clone() 


### Q-Network Architecture

In [6]:
traffic_patterns = itertools.cycle(["P1", "P2", "P3", "P4"])

def change_env():
    pattern = next(traffic_patterns)
    if traci.isLoaded():
        traci.close()
    traci.start(sumo_config(pattern))


In [7]:
def choose_action(state, epsilon, policy_net):
    if random.random() < epsilon:
        return random.randint(0, action_space_size - 1)
    else:
        return torch.argmax(policy_net(state.unsqueeze(0))).item()


In [8]:
def optimise_model(policy_net, target_net, memory, optimizer, batch_size, gamma):
    if len(memory) < batch_size:
        return
    batch = random.sample(memory, batch_size)
    states = torch.stack([x[0] for x in batch])
    actions = torch.tensor([x[1] for x in batch]).unsqueeze(1)
    rewards = torch.tensor([x[2] for x in batch], dtype=torch.float)
    next_states = torch.stack([x[3] for x in batch])
    dones = torch.tensor([x[4] for x in batch], dtype=torch.float)

    q_vals = policy_net(states).gather(1, actions).squeeze()
    with torch.no_grad():
        max_next_q_vals = target_net(next_states).max(1)[0]
        target_vals = rewards + gamma * max_next_q_vals * (1 - dones)
    loss = nn.MSELoss()(q_vals, target_vals)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


## **Baseline Model: Mimicing TrafficLightsNSW**

In [9]:

def get_baseline_results(episodes=40):
    pattern_names = ["P1", "P2", "P3", "P4"]
    pattern_name = itertools.cycle(pattern_names)
    pattern_episode_count = collections.defaultdict(int)
    pattern_rewards = collections.defaultdict(list)
    pattern_avg_waits = collections.defaultdict(list)
    pattern_max_waits = collections.defaultdict(list)
    pattern_queue_lengths = {p: [] for p in pattern_names}

    baseline_all_avg_queue_lengths = torch.zeros(len(lane_detectors), episodes)

    phase_sequence = [2, 6, 4, 0]
    green_duration = DELTA_PHASE_DURATION
    yellow_duration = YELLOW_PHASE_DURATION

    for episode in range(episodes):
        print(f"[Baseline] Episode {episode}")
        current_pattern = next(pattern_name)
        pattern_episode_count[current_pattern] += 1

        change_env()

        episode_reward = 0
        done = False
        num_steps = 0

        vehicle_wait_tracker = {}
        queue_length_tracker = {}

        state = get_current_state()
        phase_index = 0

        while not done:
            green_phase = phase_sequence[phase_index]
            yellow_phase = green_phase + 1

            # Green
            traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, green_phase)
            simulate_time(green_duration)
            num_steps += green_duration * 20

            # Reward
            state = get_current_state()
            queue_size = torch.sum(state)
            reward = -queue_size
            episode_reward += reward

            # Wait tracking
            for v_id in traci.vehicle.getIDList():
                wait_time = traci.vehicle.getWaitingTime(v_id)
                if v_id not in vehicle_wait_tracker or wait_time > vehicle_wait_tracker[v_id]:
                    vehicle_wait_tracker[v_id] = wait_time

            # Queue length tracking
            curr_queue = get_queue_length()
            print(curr_queue)
            for i in range(len(curr_queue)):
                queue_length_tracker[i] = queue_length_tracker.get(i, 0) + curr_queue[i]

            done = traci.simulation.getMinExpectedNumber() == 0
            if done:
                break

            # Yellow
            traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, yellow_phase)
            simulate_time(yellow_duration)
            num_steps += yellow_duration * 20

            done = traci.simulation.getMinExpectedNumber() == 0
            phase_index = (phase_index + 1) % len(phase_sequence)

        print("Steps per episode:", num_steps)

        avg_queue_lengths = [queue_length_tracker.get(i, 0.0) / num_steps for i in range(len(lane_detectors))]
        pattern_queue_lengths[current_pattern].append(avg_queue_lengths)

        vehicle_waits = list(vehicle_wait_tracker.values())
        avg_wait = sum(vehicle_waits) / len(vehicle_waits) if vehicle_waits else 0.0
        max_wait = max(vehicle_waits) if vehicle_waits else 0.0

        pattern_rewards[current_pattern].append(episode_reward)
        pattern_avg_waits[current_pattern].append(avg_wait)
        pattern_max_waits[current_pattern].append(max_wait)

        print(torch.sum(baseline_all_avg_queue_lengths[:, episode]))

    for pattern in pattern_names:
        print(f"\nPattern {pattern}:")
        print(f"  Episodes: {pattern_episode_count[pattern]}")
        print(f"  Avg Reward: {sum(pattern_rewards[pattern]) / len(pattern_rewards[pattern]):.2f}")
        print(f"  Avg Wait: {sum(pattern_avg_waits[pattern]) / len(pattern_avg_waits[pattern]):.2f}")
        print(f"  Max Wait: {max(pattern_max_waits[pattern]):.2f}")

    return (
        pattern_rewards,
        pattern_avg_waits,
        pattern_max_waits,
        pattern_queue_lengths
    )


In [10]:
baseline_results = get_baseline_results(episodes=40)
baseline_rewards, baseline_avg_waits, baseline_max_waits, baseline_queues = baseline_results


[Baseline] Episode 0
tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 1., 0., 0., 0., 1., 0.])
tensor([1., 0., 2., 1., 1., 0., 1., 1.])
tensor([0., 0., 1., 2., 0., 0., 2., 1.])
tensor([2., 0., 0., 1., 0., 0., 0., 0.])
tensor([2., 2., 1., 1., 0., 0., 3., 1.])
tensor([2., 1., 2., 3., 0., 0., 3., 2.])
tensor([0., 0., 1., 3., 0., 0., 4., 3.])
tensor([2., 0., 0., 0., 1., 0., 0., 1.])
tensor([2., 0., 1., 0., 1., 1., 1., 1.])
tensor([3., 0., 1., 2., 1., 1., 2., 1.])
tensor([0., 0., 5., 2., 0., 0., 3., 2.])
tensor([1., 0., 0., 0., 1., 0., 0., 2.])
tensor([1., 0., 1., 0., 1., 1., 1., 0.])
tensor([3., 0., 1., 2., 2., 1., 2., 2.])
tensor([0., 0., 1., 2., 0., 0., 3., 2.])
tensor([0., 0., 0., 1., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 1., 0.])
tensor([1., 0., 0., 0., 0., 0., 2., 1.])
tensor([0., 0., 1., 1., 0., 0., 2., 1.])
tensor([0., 0., 0., 2., 0., 0., 0., 1.])
tensor([2., 0., 1., 0., 1., 1., 1., 3.])
tensor([2., 1., 3., 2., 2., 1., 1., 3.])
tensor([0., 0., 3., 2., 0., 0., 1., 

## **Architecture 1: Fully Connected Q-Network**

In [34]:
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(state_dim, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, action_dim)
        )

    def forward(self, x):
        return self.main(x)


In [35]:
# The current state is defined as queue length for each lane detector
def get_current_state():
    return get_queue_length()


In [13]:
# Add per-lane metrics
def train_algorithm(params, episodes=200):
    gamma = params["gamma"]
    epsilon = params["epsilon"]
    epsilon_decay = params["epsilon_decay"]
    min_epsilon = params["min_epsilon"]
    lr = params["learning_rate"]
    batch_size = params["batch_size"]
    target_update_freq = params["target_update_freq"]
    memory_size = params["memory_size"]

    state_dim = len(lane_detectors)
    policy_net = DQN(state_dim, action_space_size)
    target_net = DQN(state_dim, action_space_size)
    target_net.load_state_dict(policy_net.state_dict())
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)
    memory = deque(maxlen=memory_size)

    rewards_per_episode = []
    avg_wait_per_ep = []
    max_wait_per_ep = []
    all_avg_queue_lengths = torch.zeros(len(lane_detectors), episodes)
    steps_done = 0

    pattern_names = ["P1", "P2", "P3", "P4"]
    pattern_name = itertools.cycle(pattern_names)
    pattern_episode_count = collections.defaultdict(int)
    pattern_rewards = collections.defaultdict(list)
    pattern_avg_waits = collections.defaultdict(list)
    pattern_max_waits = collections.defaultdict(list)

    for episode in range(episodes):
        change_env()
        current_pattern = next(pattern_name)
        pattern_episode_count[current_pattern] += 1

        state = get_current_state()
        episode_reward = 0
        done = False
        vehicle_wait_tracker = {}
        queue_length_tracker = {}
        num_steps = 0

        while not done:
            action = choose_action(state, epsilon, policy_net)
            next_state, reward, done, curr_queue = step(action)
            memory.append((state, action, reward, next_state, done))
            state = next_state
            episode_reward += reward

            # Vehicle wait time tracking
            for v_id in traci.vehicle.getIDList():
                wait_time = traci.vehicle.getWaitingTime(v_id)
                if v_id not in vehicle_wait_tracker:
                    vehicle_wait_tracker[v_id] = wait_time
                elif wait_time > vehicle_wait_tracker[v_id]:
                    vehicle_wait_tracker[v_id] = wait_time

            # Queue length tracking
            for i in range(len(curr_queue)):
                queue_length_tracker[i] = queue_length_tracker.get(i, 0) + curr_queue[i]

            optimise_model(policy_net, target_net, memory, optimizer, batch_size, gamma)
            if steps_done % target_update_freq == 0:
                target_net.load_state_dict(policy_net.state_dict())
            steps_done += 1
            num_steps += 1

        # Aggregate queue stats
        for i, total_len in queue_length_tracker.items():
            all_avg_queue_lengths[i, episode] = total_len / num_steps

        vehicle_waits = list(vehicle_wait_tracker.values())
        avg_wait = sum(vehicle_waits) / len(vehicle_waits) if vehicle_waits else 0.0
        max_wait = max(vehicle_waits) if vehicle_waits else 0.0
        avg_wait_per_ep.append(avg_wait)
        max_wait_per_ep.append(max_wait)

        epsilon = max(min_epsilon, epsilon * epsilon_decay)
        rewards_per_episode.append(episode_reward.item())

        pattern_rewards[current_pattern].append(episode_reward.item())
        pattern_avg_waits[current_pattern].append(avg_wait)
        pattern_max_waits[current_pattern].append(max_wait)

    N = max(1, int(0.2 * episodes))  # 20% of episodes
    avg_reward_last_N = sum(rewards_per_episode[-N:]) / N
    avg_wait_last_N = sum(avg_wait_per_ep[-N:]) / N
    max_wait_last_N = max(max_wait_per_ep[-N:]) if max_wait_per_ep[-N:] else 0.0

    return {
        "avg_reward": sum(rewards_per_episode) / episodes,
        "avg_wait_per_ep": avg_wait_per_ep,
        "max_wait_per_ep": max_wait_per_ep,
        "avg_queue_lengths": all_avg_queue_lengths,
        "rewards_per_episode": rewards_per_episode,
        "avg_reward_last_N": avg_reward_last_N,
        "avg_wait_last_N": avg_wait_last_N,
        "max_wait_last_N": max_wait_last_N,
        "trained_model": policy_net,
        "pattern_rewards": pattern_rewards,
        "pattern_avg_waits": pattern_avg_waits,
        "pattern_max_waits": pattern_max_waits
    }


In [14]:
optimal_params = {
    "gamma": 0.99,
    "epsilon": 0.9,
    "epsilon_decay": 0.95,
    "min_epsilon": 0.05,
    "learning_rate": 0.001,
    "batch_size": 128,
    "target_update_freq": 1800,
    "memory_size": 20000
}


In [15]:
fc_results = train_algorithm(optimal_params, episodes=200)


# COULD INSTEAD USE FC_RESULTS AS SOME PLOTS TO INDICATE HOW REWARD CHANGES WITH TRAINING OR SMTH


# print("### Fully Connected Q-Network Results ###")
# for pattern in ["P1", "P2", "P3", "P4"]:
#     rewards = fc_results["pattern_rewards"][pattern]
#     avg_waits = fc_results["pattern_avg_waits"][pattern]
#     max_waits = fc_results["pattern_max_waits"][pattern]

#     print(f"\nPattern {pattern}:")
#     print(f" Episodes: {len(rewards)}")
#     print(f" Avg Reward: {sum(rewards) / len(rewards):.2f}")
#     print(f" Avg Wait: {sum(avg_waits) / len(avg_waits):.2f}")
#     print(f" Max Wait: {max(max_waits):.2f}")


In [16]:
policy_fc = fc_results["trained_model"]
model_path = f"policy_fc_nn.pt"
torch.save(policy_fc.state_dict(), model_path)


In [17]:
def evaluate_fc_model_on_pattern(pattern, model_path=f"policy_fc_nn.pt", episodes=10):
    policy_net = DQN(len(lane_detectors), action_space_size)
    policy_net.load_state_dict(torch.load(model_path))
    policy_net.eval()

    rewards_per_episode = []
    avg_wait_per_ep = []
    max_wait_per_ep = []
    all_avg_queue_lengths = torch.zeros(len(lane_detectors), episodes)

    pattern_rewards = []
    pattern_avg_waits = []
    pattern_max_waits = []

    for episode in range(episodes):
        reset_env(pattern)
        state = get_current_state()

        episode_reward = 0
        done = False
        vehicle_wait_tracker = {}
        queue_length_tracker = {}
        num_steps = 0

        while not done:
            with torch.no_grad():
                action = policy_net(torch.tensor(state, dtype=torch.float32).unsqueeze(0)).argmax().item()
                
            next_state, reward, done, curr_queue = step(action)
            state = next_state
            episode_reward += reward

            # Vehicle wait time tracking
            for v_id in traci.vehicle.getIDList():
                wait_time = traci.vehicle.getWaitingTime(v_id)
                if v_id not in vehicle_wait_tracker:
                    vehicle_wait_tracker[v_id] = wait_time
                elif wait_time > vehicle_wait_tracker[v_id]:
                    vehicle_wait_tracker[v_id] = wait_time

            # Queue length tracking
            for i in range(len(curr_queue)):
                queue_length_tracker[i] = queue_length_tracker.get(i, 0) + curr_queue[i]

            num_steps += 1

        for i, total_len in queue_length_tracker.items():
            all_avg_queue_lengths[i, episode] = total_len / num_steps

        vehicle_waits = list(vehicle_wait_tracker.values())
        avg_wait = sum(vehicle_waits) / len(vehicle_waits) if vehicle_waits else 0.0
        max_wait = max(vehicle_waits) if vehicle_waits else 0.0

        avg_wait_per_ep.append(avg_wait)
        max_wait_per_ep.append(max_wait)
        rewards_per_episode.append(episode_reward.item())

        pattern_rewards.append(episode_reward.item())
        pattern_avg_waits.append(avg_wait)
        pattern_max_waits.append(max_wait)

    return {
        "pattern": pattern,
        "avg_reward": sum(rewards_per_episode) / episodes,
        "avg_wait_per_ep": avg_wait_per_ep,
        "max_wait_per_ep": max_wait_per_ep,
        "avg_queue_lengths": all_avg_queue_lengths,
        "rewards_per_episode": rewards_per_episode,
        "pattern_rewards": pattern_rewards,
        "pattern_avg_waits": pattern_avg_waits,
        "pattern_max_waits": pattern_max_waits
    }


In [38]:
for pattern in ["P1", "P2", "P3", "P4"]:
    fc_results = evaluate_fc_model_on_pattern(pattern, episodes=1)
    rewards = fc_results["pattern_rewards"]
    avg_waits = fc_results["pattern_avg_waits"]
    max_waits = fc_results["pattern_max_waits"]

    print(f"\nPattern {pattern}:")
    print(f" Episodes: {len(rewards)}")
    print(f" Avg Reward: {sum(rewards) / len(rewards):.2f}")
    print(f" Avg Wait: {sum(avg_waits) / len(avg_waits):.2f}")
    print(f" Max Wait: {max(max_waits):.2f}")


C:\Users\alexw\AppData\Local\Temp\ipykernel_19104\3986529690.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy_net.load_state_dict(torch.load(model_path))
C:\Users\


Pattern P1:
 Episodes: 1
 Avg Reward: -1126.00
 Avg Wait: 7.48
 Max Wait: 90.65

Pattern P2:
 Episodes: 1
 Avg Reward: -1415.00
 Avg Wait: 9.02
 Max Wait: 214.50

Pattern P3:
 Episodes: 1
 Avg Reward: -1540.00
 Avg Wait: 9.04
 Max Wait: 128.30

Pattern P4:
 Episodes: 1
 Avg Reward: -14.00
 Avg Wait: 0.00
 Max Wait: 0.00


In [ ]:
print(fc_results)


{'pattern': 'P4', 'avg_reward': -14.0, 'avg_wait_per_ep': [0.0], 'max_wait_per_ep': [0.0], 'avg_queue_lengths': tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0495],
        [0.0000]]), 'rewards_per_episode': [-14.0], 'pattern_rewards': [-14.0], 'pattern_avg_waits': [0.0], 'pattern_max_waits': [0.0]}


## **Architecture 2: CNN Q-Network**

In [19]:
class DQN(nn.Module):
    def __init__(self, num_actions):
        super(DQN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=32, kernel_size=8, stride=4)  
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)  
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)  
        
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        self.output = nn.Linear(512, num_actions)

    def forward(self, x):
        x = x / 255.0  
        x = F.relu(self.conv1(x))   
        x = F.relu(self.conv2(x))  
        x = F.relu(self.conv3(x))   
        
        x = x.view(x.size(0), -1)   
        x = F.relu(self.fc1(x))
        return self.output(x)      


In [20]:
def generate_occupancy_grid(grid_size=(84, 84),
                             bounds=(-115, 100, -90, 125)):
    x_min, x_max, y_min, y_max = bounds
    x_scale = grid_size[1] / (x_max - x_min)
    y_scale = grid_size[0] / (y_max - y_min)

    grid = torch.zeros(grid_size, dtype=torch.float32)

    for v_id in traci.vehicle.getIDList():
        x, y = traci.vehicle.getPosition(v_id)

        if x_min <= x <= x_max and y_min <= y <= y_max:
            col = int((x - x_min) * x_scale)
            row = int((y - y_min) * y_scale)

            if 0 <= row < grid_size[0] and 0 <= col < grid_size[1]:
                grid[row, col] = 1.0  

    return grid

frame_buffer = deque(maxlen=4)

def get_current_state():
    frame_buffer.clear()  # clear previous frames to maintain consistency

    for _ in range(4):
        simulate_time(1)  # simulate 1 second
        grid = generate_occupancy_grid()
        frame_buffer.append(grid.unsqueeze(0))

    state = torch.cat(list(frame_buffer), dim=0)
    return state


def step(action):
    global current_phase
    if 2 * action == current_phase:
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, 2 * action)
        simulate_time(DELTA_PHASE_DURATION - 4)
    else:
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, current_phase + 1)
        simulate_time(YELLOW_PHASE_DURATION)
        current_phase = 2 * action
        traci.trafficlight.setPhase(TRAFFIC_LIGHT_ID, 2 * action)
        simulate_time(DELTA_PHASE_DURATION - 4)
    next_state = get_current_state()
    queue_length = get_queue_length()
    reward = -torch.sum(queue_length)
    done = traci.simulation.getMinExpectedNumber() == 0
    return next_state, reward, done, queue_length


In [21]:
# Add per-lane metrics
def train_algorithm(params, episodes=200):
    gamma = params["gamma"]
    epsilon = params["epsilon"]
    epsilon_decay = params["epsilon_decay"]
    min_epsilon = params["min_epsilon"]
    lr = params["learning_rate"]
    batch_size = params["batch_size"]
    target_update_freq = params["target_update_freq"]
    memory_size = params["memory_size"]

    policy_net = DQN(action_space_size)
    target_net = DQN(action_space_size)
    target_net.load_state_dict(policy_net.state_dict())
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)
    memory = deque(maxlen=memory_size)

    rewards_per_episode = []
    avg_wait_per_ep = []
    max_wait_per_ep = []
    all_avg_queue_lengths = torch.zeros(len(lane_detectors), episodes)
    steps_done = 0

    pattern_names = ["P1", "P2", "P3", "P4"]
    pattern_name = itertools.cycle(pattern_names)
    pattern_episode_count = collections.defaultdict(int)
    pattern_rewards = collections.defaultdict(list)
    pattern_avg_waits = collections.defaultdict(list)
    pattern_max_waits = collections.defaultdict(list)

    for episode in range(episodes):
        change_env()
        current_pattern = next(pattern_name)
        pattern_episode_count[current_pattern] += 1

        state = get_current_state()
        episode_reward = 0
        done = False
        vehicle_wait_tracker = {}
        queue_length_tracker = {}
        num_steps = 0

        while not done:
            action = choose_action(state, epsilon, policy_net)
            next_state, reward, done, curr_queue = step(action)
            memory.append((state, action, reward, next_state, done))
            state = next_state
            episode_reward += reward

            for v_id in traci.vehicle.getIDList():
                wait_time = traci.vehicle.getWaitingTime(v_id)
                if v_id not in vehicle_wait_tracker:
                    vehicle_wait_tracker[v_id] = wait_time
                elif wait_time > vehicle_wait_tracker[v_id]:
                    vehicle_wait_tracker[v_id] = wait_time

            for i in range(len(curr_queue)):
                queue_length_tracker[i] = queue_length_tracker.get(i, 0) + curr_queue[i]

            optimise_model(policy_net, target_net, memory, optimizer, batch_size, gamma)
            if steps_done % target_update_freq == 0:
                target_net.load_state_dict(policy_net.state_dict())
            steps_done += 1
            num_steps += 1

        for i, total_len in queue_length_tracker.items():
            all_avg_queue_lengths[i, episode] = total_len / num_steps

        vehicle_waits = list(vehicle_wait_tracker.values())
        avg_wait = sum(vehicle_waits) / len(vehicle_waits) if vehicle_waits else 0.0
        max_wait = max(vehicle_waits) if vehicle_waits else 0.0
        avg_wait_per_ep.append(avg_wait)
        max_wait_per_ep.append(max_wait)

        epsilon = max(min_epsilon, epsilon * epsilon_decay)
        rewards_per_episode.append(episode_reward.item())

        pattern_rewards[current_pattern].append(episode_reward.item())
        pattern_avg_waits[current_pattern].append(avg_wait)
        pattern_max_waits[current_pattern].append(max_wait)

        print(f"[CNN] Episode {episode}: Reward {episode_reward.item():.2f}")

    N = max(1, int(0.2 * episodes))
    avg_reward_last_N = sum(rewards_per_episode[-N:]) / N
    avg_wait_last_N = sum(avg_wait_per_ep[-N:]) / N
    max_wait_last_N = max(max_wait_per_ep[-N:]) if max_wait_per_ep[-N:] else 0.0

    return {
        "avg_reward": sum(rewards_per_episode) / episodes,
        "avg_wait_per_ep": avg_wait_per_ep,
        "max_wait_per_ep": max_wait_per_ep,
        "avg_queue_lengths": all_avg_queue_lengths,
        "rewards_per_episode": rewards_per_episode,
        "avg_reward_last_N": avg_reward_last_N,
        "avg_wait_last_N": avg_wait_last_N,
        "max_wait_last_N": max_wait_last_N,
        "trained_model": policy_net,
        "pattern_rewards": pattern_rewards,
        "pattern_avg_waits": pattern_avg_waits,
        "pattern_max_waits": pattern_max_waits
    }


In [22]:
optimal_params = {
    "gamma": 0.999,
    "epsilon": 0.9,
    "epsilon_decay": 0.95,
    "min_epsilon": 0.05,
    "learning_rate": 0.001,
    "batch_size": 128,
    "target_update_freq": 1800,
    "memory_size": 20000
}


In [23]:
cnn_results = train_algorithm(optimal_params, episodes=200)
# print("\n### CNN Q-Network Results ###")

# for pattern in ["P1", "P2", "P3", "P4"]:
#     rewards = cnn_results["pattern_rewards"][pattern]
#     avg_waits = cnn_results["pattern_avg_waits"][pattern]
#     max_waits = cnn_results["pattern_max_waits"][pattern]

#     if len(rewards) == 0:
#         print(f"\nPattern {pattern}: No episodes run.")
#         continue

#     print(f"\nPattern {pattern}:")
#     print(f" Episodes: {len(rewards)}")
#     print(f" Avg Reward: {sum(rewards) / len(rewards):.2f}")
#     print(f" Avg Wait: {sum(avg_waits) / len(avg_waits):.2f}")
#     print(f" Max Wait: {max(max_waits):.2f}")


[CNN] Episode 0: Reward -6143.00
[CNN] Episode 1: Reward -6788.00
[CNN] Episode 2: Reward -5978.00
[CNN] Episode 3: Reward -252.00
[CNN] Episode 4: Reward -3256.00
[CNN] Episode 5: Reward -5092.00
[CNN] Episode 6: Reward -3500.00
[CNN] Episode 7: Reward -91.00
[CNN] Episode 8: Reward -4064.00
[CNN] Episode 9: Reward -3929.00
[CNN] Episode 10: Reward -3535.00
[CNN] Episode 11: Reward -129.00
[CNN] Episode 12: Reward -2124.00
[CNN] Episode 13: Reward -2077.00
[CNN] Episode 14: Reward -2340.00
[CNN] Episode 15: Reward -81.00
[CNN] Episode 16: Reward -1874.00
[CNN] Episode 17: Reward -3599.00
[CNN] Episode 18: Reward -2098.00
[CNN] Episode 19: Reward -74.00
[CNN] Episode 20: Reward -1383.00
[CNN] Episode 21: Reward -2119.00
[CNN] Episode 22: Reward -1934.00
[CNN] Episode 23: Reward -55.00
[CNN] Episode 24: Reward -1521.00
[CNN] Episode 25: Reward -1954.00
[CNN] Episode 26: Reward -3251.00
[CNN] Episode 27: Reward -56.00
[CNN] Episode 28: Reward -1655.00
[CNN] Episode 29: Reward -1642.00
[C

In [24]:
policy_cnn = cnn_results["trained_model"]
model_path = f"policy_cnn_nn.pt"
torch.save(policy_cnn.state_dict(), model_path)


In [31]:
def evaluate_cnn_model_on_pattern(pattern, model_path=f"policy_cnn_nn.pt", episodes=10):
    policy_net = DQN(action_space_size)
    policy_net.load_state_dict(torch.load(model_path))
    policy_net.eval()

    rewards_per_episode = []
    avg_wait_per_ep = []
    max_wait_per_ep = []
    all_avg_queue_lengths = torch.zeros(len(lane_detectors), episodes)

    pattern_rewards = []
    pattern_avg_waits = []
    pattern_max_waits = []

    for episode in range(episodes):
        reset_env(pattern)
        state = get_current_state()

        episode_reward = 0
        done = False
        vehicle_wait_tracker = {}
        queue_length_tracker = {}
        num_steps = 0

        while not done:
            with torch.no_grad():
                action = policy_net(torch.tensor(state, dtype=torch.float32).unsqueeze(0)).argmax().item()
                
            next_state, reward, done, curr_queue = step(action)
            state = next_state
            episode_reward += reward

            # Vehicle wait time tracking
            for v_id in traci.vehicle.getIDList():
                wait_time = traci.vehicle.getWaitingTime(v_id)
                if v_id not in vehicle_wait_tracker:
                    vehicle_wait_tracker[v_id] = wait_time
                elif wait_time > vehicle_wait_tracker[v_id]:
                    vehicle_wait_tracker[v_id] = wait_time

            # Queue length tracking
            for i in range(len(curr_queue)):
                queue_length_tracker[i] = queue_length_tracker.get(i, 0) + curr_queue[i]

            num_steps += 1

        for i, total_len in queue_length_tracker.items():
            all_avg_queue_lengths[i, episode] = total_len / num_steps

        vehicle_waits = list(vehicle_wait_tracker.values())
        avg_wait = sum(vehicle_waits) / len(vehicle_waits) if vehicle_waits else 0.0
        max_wait = max(vehicle_waits) if vehicle_waits else 0.0

        avg_wait_per_ep.append(avg_wait)
        max_wait_per_ep.append(max_wait)
        rewards_per_episode.append(episode_reward.item())

        pattern_rewards.append(episode_reward.item())
        pattern_avg_waits.append(avg_wait)
        pattern_max_waits.append(max_wait)

    return {
        "pattern": pattern,
        "avg_reward": sum(rewards_per_episode) / episodes,
        "avg_wait_per_ep": avg_wait_per_ep,
        "max_wait_per_ep": max_wait_per_ep,
        "avg_queue_lengths": all_avg_queue_lengths,
        "rewards_per_episode": rewards_per_episode,
        "pattern_rewards": pattern_rewards,
        "pattern_avg_waits": pattern_avg_waits,
        "pattern_max_waits": pattern_max_waits
    }


In [32]:
for pattern in ["P1", "P2", "P3", "P4"]:
    cnn_results = evaluate_cnn_model_on_pattern(pattern, episodes=1)
    rewards = cnn_results["pattern_rewards"]
    avg_waits = cnn_results["pattern_avg_waits"]
    max_waits = cnn_results["pattern_max_waits"]

    print(f"\nPattern {pattern}:")
    print(f" Episodes: {len(rewards)}")
    print(f" Avg Reward: {sum(rewards) / len(rewards):.2f}")
    print(f" Avg Wait: {sum(avg_waits) / len(avg_waits):.2f}")
    print(f" Max Wait: {max(max_waits):.2f}")


C:\Users\alexw\AppData\Local\Temp\ipykernel_19104\3032803598.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy_net.load_state_dict(torch.load(model_path))
C:\Users\


Pattern P1:
 Episodes: 1
 Avg Reward: -1403.00
 Avg Wait: 9.72
 Max Wait: 298.00

Pattern P2:
 Episodes: 1
 Avg Reward: -1541.00
 Avg Wait: 10.84
 Max Wait: 145.20

Pattern P3:
 Episodes: 1
 Avg Reward: -1746.00
 Avg Wait: 11.19
 Max Wait: 142.60

Pattern P4:
 Episodes: 1
 Avg Reward: -39.00
 Avg Wait: 0.82
 Max Wait: 13.70
